In [1]:
import data.sqlDataFetch as sdf

In [2]:
id=sdf.get_id_by_name('APPROVED BUILDING PERMITS')
print(id)
sdf.describe_database(id)

6ddcd912-32a0-43df-9908-63574f8c7e77
Column Name          | Type      
--------------------------------
_id                  | int       
permitnumber         | text      
worktype             | text      
permittypedescr      | text      
description          | text      
comments             | text      
applicant            | text      
declared_valuation   | text      
total_fees           | text      
issued_date          | text      
expiration_date      | text      
status               | text      
owner                | text      
occupancytype        | text      
sq_feet              | text      
address              | text      
city                 | text      
state                | text      
zip                  | text      
property_id          | text      
parcel_id            | text      
gpsy                 | text      
gpsx                 | text      
geom_point_2249      | text      
geom_point_4326      | text      


In [4]:
import requests

url = "https://data.boston.gov/api/3/action/datastore_search_sql"
# Construct the SQL query to select distinct city names from the table
sql_query = f"SELECT DISTINCT city FROM \"6ddcd912-32a0-43df-9908-63574f8c7e77\""

params = {"sql": sql_query}

# Send the request
response = requests.get(url, params=params)

allCitiesName = []

if response.status_code == 200:
    data = response.json()

    # Check if there is data in the response
    if data['success'] and 'result' in data and 'records' in data['result']:
        records = data['result']['records']
        # Loop through each record in the response
        for record in records:
          # Check if the city value is not None
          if record['city'] is not None:
            # Add the city name to the list
            allCitiesName.append(record['city'])
    else:
        print("No data found or error in response.")
else:
    print("Failed to fetch data:", response.status_code)

In [5]:
print(allCitiesName)

['North End', 'CHARLESTOWN', 'Ropxbury', 'Back /', "Mission Hill'", 'EAST BOSTON', 'Fenway', 'Financial District/', 'Brighton/Allston', 'West Roxbury/', 'Downtown/Financial District', 'Boston/Charlestown', 'Boston/Chinatown', 'Brighton/', 'BRIGHTON', 'Boston/Beacon Hill', 'South BostonSG', 'ROXBURY CROSSIN', 'roxbury', 'Boston/South End', 'Chestnut Hill', 'JAMAICA PLAIN', 'ROSLINDALE', 'Dorchester Center//', 'South End//', 'Theater District', 'ALLSTON', 'dorchester', 'Mission Hill/', 'East Boston/', 'Mission Hill', 'Charlestown/', 'Jamaica Plain/Boston', 'West End', 'Dorchester (Savin Hill)/', 'East BOSTON', 'BRIGHTON/', 'Hyde Park', 'West Roxbury', 'Hyde Park//', 'Boston/South', 'Dorchester//', 'Brighton', 'Charlestown+', 'Downtown/Financial', 'BOSTON', 'NorthEnd', 'Allston/', 'Boston/Back Bay', 'E BOSTON', 'West Robury', 'Chinatown/', 'Back Bay/', 'South End/', 'South Boston', 'Dorchester Center/', 'BOSTON/CHINATOWN', 'boston', 'East  Boston', 'BOSTON/', 'ROXBURY', 'Boston/Fenway', '

In [6]:
dfs = []

for city in allCitiesName:
    condition = f"WHERE \"city\"::text = '{city}'"
    result = sdf.fetch_data(id, condition, _id=int, permitnumber=str, worktype=str, permittypedescr=str, issued_date='time', expiration_date='time', status=str, address=str, city=str, zip=str, occupancytype=str, declared_valuation='money')

    if not result.empty:
        dfs.append(result)
    else:
        print(f"No data or failed to fetch data for {city}")

Failed to fetch data: 409
No data or failed to fetch data for Mission Hill'
Failed to fetch data: 409
No data or failed to fetch data for Boston's Historic North End


In [7]:
condition = "WHERE \"city\"::text = 'Mission Hill'"
result = sdf.fetch_data(id, condition, _id=int, permitnumber=str, worktype=str, permittypedescr=str, issued_date='time', expiration_date='time', status=str, address=str, city=str, zip=str, occupancytype=str, declared_valuation='money')

if not result.empty:
  dfs.append(result)
else:
  print("No data or failed to fetch data for Mission Hill")

In [8]:
condition = "WHERE \"city\"::text = 'Boston''s Historic North End'"
result = sdf.fetch_data(id, condition, _id=int, permitnumber=str, worktype=str, permittypedescr=str, issued_date='time', expiration_date='time', status=str, address=str, city=str, zip=str, occupancytype=str, declared_valuation='money')

if not result.empty:
  dfs.append(result)
else:
  print("No data or failed to fetch data for Boston\'s Historic North End")

In [9]:
import pandas as pd

combined_df = pd.concat(dfs, ignore_index=True)

In [10]:
print(f"Combined DataFrame has {len(combined_df)} rows.")

Combined DataFrame has 374140 rows.


In [11]:
print(combined_df)

           _id permitnumber  worktype              permittypedescr  \
0       549377      SF48002    INTEXT       Short Form Bldg Permit   
1       451679     SF105658    EXTREN       Short Form Bldg Permit   
2       513033      SF20887    EXTREN       Short Form Bldg Permit   
3        24400      ALT3406    EXTREN  Long Form/Alteration Permit   
4       319516       G16827       COB                   Gas Permit   
...        ...          ...       ...                          ...   
374135   68246    COO933462    NEWCON     Certificate of Occupancy   
374136  441458     PL941436  PLUMBING              Plumbing Permit   
374137   67757    COO906874    NEWCON     Certificate of Occupancy   
374138  287729    ERT263467    NEWCON       Erect/New Construction   
374139  609053     SF935876    INTREN       Short Form Bldg Permit   

                   issued_date         expiration_date  status  \
0       2010-10-29 17:40:28+00  2011-04-29 04:00:00+00    Open   
1       2011-11-16 14:47:14

In [14]:
import os

save_path = './data/cleanedData/'
file_name = 'approved_building permits_data.csv'
full_path = os.path.join(save_path, file_name)

if not os.path.exists(save_path):
  os.makedirs(save_path)

combined_df.to_csv(full_path, index=False)